# Global averaging

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.signal import detrend
from matplotlib import pyplot as plt
from scipy import signal
import pandas as pd
import xarray as xr
import intake
import pprint 
import util 
import os

col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)

file = 'available_tas_data_cloud_june12th_2020.txt'

In [2]:
# load table:
data_table = pd.read_table(file,index_col=0)
models_used = data_table['piControl (yrs)'].index
print(models_used)

Index(['IPSL-CM6A-LR', 'MRI-ESM2-0', 'MPI-ESM1-2-LR', 'TaiESM1',
       'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'BCC-ESM1',
       'CAMS-CSM1-0', 'FGOALS-f3-L', 'FGOALS-g3', 'IITM-ESM', 'CanESM5',
       'CanESM5-CanOE', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1',
       'ACCESS-ESM1-5', 'ACCESS-CM2', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA',
       'EC-Earth3', 'EC-Earth3-LR', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR',
       'FIO-ESM-2-0', 'MPI-ESM-1-2-HAM', 'INM-CM4-8', 'INM-CM5-0',
       'MIROC-ES2L', 'MIROC6', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM',
       'UKESM1-0-LL', 'MPI-ESM1-2-HR', 'GISS-E2-1-G', 'GISS-E2-1-G-CC',
       'GISS-E2-1-H', 'GISS-E2-2-G', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM',
       'CESM2-WACCM-FV2', 'NorCPM1', 'NorESM1-F', 'NorESM2-LM', 'NorESM2-MM',
       'KACE-1-0-G', 'GFDL-CM4', 'GFDL-ESM4', 'NESM3', 'SAM0-UNICON', 'CIESM',
       'MCM-UA-1-0'],
      dtype='object')


In [3]:
# load functions:
def area_weights(lat_bnds, lon_bnds): 
    # computes exact area weigths assuming earth is a perfect sphere
    lowerlats = np.radians(lat_bnds[:,0]); upperlats = np.radians(lat_bnds[:,1])
    difflon = np.radians(np.diff(lon_bnds[0,:])) # if the differences in longitudes are all the same
    areaweights = difflon*(np.sin(upperlats) - np.sin(lowerlats));
    areaweights /= areaweights.mean()
    return areaweights # list of weights, of same dimension as latitude

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def leap_year(year, calendar='standard'):
    """Determine if year is a leap year"""
    leap = False
    if ((calendar in ['standard', 'gregorian',
        'proleptic_gregorian', 'julian']) and
        (year % 4 == 0)):
        leap = True
        if ((calendar == 'proleptic_gregorian') and
            (year % 100 == 0) and
            (year % 400 != 0)):
            leap = False
        elif ((calendar in ['standard', 'gregorian']) and
                 (year % 100 == 0) and (year % 400 != 0) and
                 (year < 1583)):
            leap = False
    return leap

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def get_dpm(time, calendar='standard'):
    """
    return a array of days per month corresponding to the months provided in `months`
    """
    month_length = np.zeros(len(time), dtype=np.int)

    cal_days = dpm[calendar]

    for i, (month, year) in enumerate(zip(time.month, time.year)):
        month_length[i] = cal_days[month]
        if leap_year(year, calendar=calendar) and month == 2:
            month_length[i] += 1
    return month_length

# days per month:
dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'julian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31], ##### I think this should be correct
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
      }

def compute_day_weights(ds, calendar = 'noleap'): # new function
    month_length = xr.DataArray(get_dpm((ds.time.to_index()), calendar=ds_calendar), coords=[ds.time], name='month_length')
    
    ##### This code is only tested for noleap so far #####
    norm_by_annual = month_length.groupby('time.year').mean('time') # make annual mean
    norm_by_monthly = np.concatenate([np.tile(norm_by_annual.values[i], 12) for i in range(len(norm_by_annual.values))])
    weights = month_length/norm_by_monthly
    # normalized to have mean 1
    return weights 

def calendar_check(model):
    # Time formats for piControl, found from manual check:
    if model in ['TaiESM1', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'FGOALS-f3-L', 'FGOALS-g3', 'CanESM5', 'CanESM5-CanOE', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA', 'FIO-ESM-2-0', 'INM-CM4-8', 'INM-CM5-0', 'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'GISS-E2-1-H', 'GISS-E2-2-G', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'NorCPM1', 'NorESM1-F', 'NorESM2-LM', 'NorESM2-MM', 'GFDL-CM4', 'SAM0-UNICON', 'GFDL-ESM4', 'CIESM', 'MCM-UA-1-0']:
        ds_calendar = 'noleap'
    elif model in ['CNRM-CM6-1', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6', 'NESM3']:
        ds_calendar = 'gregorian'
    elif model in ['AWI-CM-1-1-MR', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'ACCESS-ESM1-5', 'ACCESS-CM2', 'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-LR', 'MPI-ESM1-2-HR']:
        ds_calendar = 'proleptic_gregorian'
    elif model in ['UKESM1-0-LL', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'CNRM-ESM2-1', 'KACE-1-0-G', 'MRI-ESM2-0']:
        ds_calendar = '360_day'
        if model in ['CNRM-ESM2-1', 'MRI-ESM2-0']:
            print('piControl is 360_day, the other experiments unknown')
    elif model in ['IITM-ESM']:
        ds_calendar = 'Julian'
    elif model in ['AWI-ESM-1-1-LR', 'CNRM-CM6-1-HR', 'EC-Earth3', 'EC-Earth3-LR']:
        #ds_calendar = 'datetime64'
        print('not 100% sure what calendar this model has, but a guess is made based on other models from same institution')
        if model in ['EC-Earth3', 'EC-Earth3-LR', 'AWI-ESM-1-1-LR']:
            print('calendar is likely proleptic gregorian')
            ds_calendar = 'proleptic_gregorian'
        elif model in ['CNRM-CM6-1-HR']:
            print('calendar is likely gregorian')
            ds_calendar = 'gregorian'
    return ds_calendar
    

# Choose model, exp, member

In [40]:
#model = models_used[10]; 
#model = 'KACE-1-0-G'
#model = 'BCC-ESM1'
#model = 'CESM2'
model = 'AWI-CM-1-1-MR'
print(model)
ds_calendar = calendar_check(model)
print(ds_calendar, 'calendar')

#check what experiments are available for var
var = 'tas'
#cat = col.search(experiment_id = ['piControl', 'abrupt-4xCO2', 'abrupt-2xCO2', 'abrupt-0p5xCO2', 'historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585'], source_id = model, variable_id=var, table_id='Amon') 
cat = col.search(experiment_id = 'piControl', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 

cat.df


AWI-CM-1-1-MR
proleptic_gregorian calendar


,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
6059,CMIP,AWI,AWI-CM-1-1-MR,piControl,r1i1p1f1,Amon,rlut,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/piControl/r1...,NaN,20191015
6063,CMIP,AWI,AWI-CM-1-1-MR,piControl,r1i1p1f1,Amon,rsdt,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/piControl/r1...,NaN,20191015
6066,CMIP,AWI,AWI-CM-1-1-MR,piControl,r1i1p1f1,Amon,rsut,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/piControl/r1...,NaN,20191015
6070,CMIP,AWI,AWI-CM-1-1-MR,piControl,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/piControl/r1...,NaN,20191015


In [41]:
#exp = 'piControl'
#exp = 'abrupt-4xCO2'
#exp = 'abrupt-2xCO2'
#exp = 'abrupt-0p5xCO2'
exp = 'historical'
#exp = 'ssp126'
#exp = 'ssp245'
#exp = 'ssp370'
#exp = 'ssp585'
#exp = 'piClim-4xCO2'
#exp = 'piClim-control'
#exp = 'piClim-histall'


if model == 'UKESM1-0-LL':
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'MOHC') 
if model == 'MPI-ESM1-2-HR': # select institution if there are two groups
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'DKRZ') 
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'DWD') 
else:
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 
    
if 'dset_dict' in locals():
    del dset_dict
if 'ds_exp' in locals():
    del ds_exp
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
for key in dset_dict.keys():
    ds_exp = dset_dict[key]
    if 'members_sorted' in locals():
        del members_sorted
    members_sorted = ds_exp.member_id.sortby(ds_exp.member_id)
    
if model == 'MCM-UA-1-0':
    ds_exp = ds_exp.rename({'longitude': 'lon','latitude': 'lat'}) 
        
for member in members_sorted:
    print(member.values)
    
# write out data variables, to check that we have all we want    
ds_exp.data_vars

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
r1i1p1f1
r2i1p1f1
r3i1p1f1
r4i1p1f1
r5i1p1f1


Data variables:
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(384, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1980, 2), meta=np.ndarray>
    rlut       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 200, 192, 384), meta=np.ndarray>
    rsdt       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 295, 192, 384), meta=np.ndarray>
    rsut       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 204, 192, 384), meta=np.ndarray>
    height     float64 ...
    tas        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 230, 192, 384), meta=np.ndarray>

In [34]:
#ds_exp.rlut.isel(time = 10).plot()
#dset_dict.keys()

In [35]:
# testing model EC earth3
#ds_test = ds_exp.sel(member_id = 'r4i1p1f1')
#ds_test['tas'].isel(time=0).values[0]
#import sys
#np.set_printoptions(threshold=sys.maxsize)
#ds_test['tas'].isel(time=0).values[0].shape
#ds_test.lon

In [42]:
# loop through members
for member in members_sorted:
    print(member.values)

    ds = ds_exp.sel(member_id = member)
    # compute weights for average
    area_w = area_weights(ds.lat_bnds.values, ds.lon_bnds.values)
    day_weights = compute_day_weights(ds)

    #if exp == 'piClim-4xCO2':
    #    varlist = ['rlut', 'rsut', 'rsdt']
    #else:
    varlist = ['tas', 'rlut', 'rsut', 'rsdt']
        
    for variable in varlist:
        print(variable)
        data = ds[variable]

        # global average
        area_avg = (data.transpose('time', 'lon', 'lat') * area_w).mean(dim=['lon', 'lat'])

        # annual average
        day_weighted_avg = area_avg*day_weights
        annualmean = day_weighted_avg.groupby('time.year').mean('time')

        if variable == varlist[0]:
            # create dataframe for storing all results
            df = pd.DataFrame(annualmean.values, columns = [variable])
        else:
            df_col = pd.DataFrame(annualmean.values, columns = [variable])
            df = pd.merge(df, df_col, left_index=True, right_index=True, how='outer')
            
    filename = model + '_' + exp + '_' + str(member.values) + '_means.txt'
    file = os.path.join('../Processed_data/Global_annual_means/', model, exp, filename)
    #if member == members_sorted[0]: # create directory for first member
    #    os.makedirs(os.path.dirname(file), exist_ok=False)

    df.to_csv(file)
    

r1i1p1f1
tas
rlut
rsut
rsdt
r2i1p1f1
tas
rlut
rsut
rsdt
r3i1p1f1
tas
rlut
rsut
rsdt
r4i1p1f1
tas
rlut
rsut
rsdt
r5i1p1f1
tas
rlut
rsut
rsdt


In [44]:
df

,tas,rlut,rsut,rsdt
0,NaN,241.297919,99.352363,340.253404
1,286.904730,241.510192,98.891892,340.243878
2,286.929121,241.670843,98.667391,340.276988
3,286.814834,241.448212,98.547663,340.220781
4,286.907293,241.688971,98.433061,340.193745
...,...,...,...,...
160,287.959925,241.000883,98.384092,340.225475
161,287.982445,240.815446,98.379966,340.291235
162,288.168558,240.777007,98.456956,340.355342
163,288.102779,240.911046,98.786678,340.333195


In [ ]:
# NorCPM1 historical: nans in lat_bnds

# Updated: NorESM, NorCPM, CESM2, GISS


In [47]:
model

'GISS-E2-1-G'

In [39]:
# check dataset availability:
#col_url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)
#cat = col.search(experiment_id = 'piClim-4xCO2', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
#cat = col.search(experiment_id = 'piClim-control', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
cat = col.search(experiment_id = 'historical', source_id = 'AWI-CM-1-1-MR', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
5781,CMIP,AWI,AWI-CM-1-1-MR,historical,r1i1p1f1,Amon,rlut,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN,20200511
5783,CMIP,AWI,AWI-CM-1-1-MR,historical,r1i1p1f1,Amon,rsdt,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN,20200511
5784,CMIP,AWI,AWI-CM-1-1-MR,historical,r1i1p1f1,Amon,rsut,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN,20200511
5788,CMIP,AWI,AWI-CM-1-1-MR,historical,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN,20200720
5838,CMIP,AWI,AWI-CM-1-1-MR,historical,r2i1p1f1,Amon,rlut,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN,20200511
5839,CMIP,AWI,AWI-CM-1-1-MR,historical,r2i1p1f1,Amon,rsdt,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN,20200511
5840,CMIP,AWI,AWI-CM-1-1-MR,historical,r2i1p1f1,Amon,rsut,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN,20200511
5843,CMIP,AWI,AWI-CM-1-1-MR,historical,r2i1p1f1,Amon,tas,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN,20200720
5889,CMIP,AWI,AWI-CM-1-1-MR,historical,r3i1p1f1,Amon,rlut,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN,20200511
5890,CMIP,AWI,AWI-CM-1-1-MR,historical,r3i1p1f1,Amon,rsdt,gn,gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r...,NaN,20200511
